In [1]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
import torch
from sklearn.svm import SVC

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout,Activation,MaxPooling2D,ReLU,Add,GlobalAvgPool2D
from tensorflow.keras import Model,Sequential
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import ast

2023-05-24 09:10:06.731773: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Get the Data and Labels

In [2]:
df_norm = pd.read_csv('data/data_prepared/data_norm_fft/data_norm.csv')

In [3]:
valence = np.load('data/data_prepared/labels/label_class_0.npy')
arousal = np.load('data/data_prepared/labels/label_class_1.npy')
dominance = np.load('data/data_prepared/labels/label_class_2.npy')
liking = np.load('data/data_prepared/labels/label_class_3.npy')

In [4]:
valence.shape

(1280,)

In [5]:
df_norm

,p:sub01 v:0,p:sub01 v:1,p:sub01 v:2,p:sub01 v:3,p:sub01 v:4,p:sub01 v:5,p:sub01 v:6,p:sub01 v:7,p:sub01 v:8,p:sub01 v:9,...,p:sub32 v:30,p:sub32 v:31,p:sub32 v:32,p:sub32 v:33,p:sub32 v:34,p:sub32 v:35,p:sub32 v:36,p:sub32 v:37,p:sub32 v:38,p:sub32 v:39
0,"[0.4485464836863576, 0.23743089577886559, 0.25...","[0.7019180566921461, 0.5461401369897261, 0.551...","[0.6288997216273957, 0.5353532230667453, 0.539...","[0.6266704748094754, 0.47343463832216404, 0.22...","[0.0, 0.027719349129351294, 0.2419823433081025...","[0.4664035232446203, 0.27838186329410847, 0.30...","[0.6703302868325047, 0.32789231857853357, 0.66...","[0.9899844902094569, 0.810246175123128, 0.8003...","[1.0, 0.8200701167617226, 0.7860376730186085, ...","[0.4504604599247114, 0.26668218946707795, 0.54...",...,"[0.023487912614575716, 0.0, 0.1688471178401590...","[0.5422144993423434, 0.5920108093137332, 0.739...","[0.7929219302890472, 0.5110720072159004, 0.446...","[0.2989311031697405, 0.39466039348987386, 0.39...","[0.383499595408718, 0.1780261984830269, 0.2179...","[0.42486739015137676, 0.38144560948409556, 0.6...","[0.8302330838047564, 0.6761027213858244, 0.686...","[0.4694137889505188, 0.3410767456862581, 0.857...","[0.3528522856935695, 0.13774190624327134, 0.37...","[0.14069406842073867, 0.06540498312891915, 0.0..."
1,"[0.4376705502296044, 0.254018240277233, 0.2693...","[0.7345198385896154, 0.4087973258721663, 0.518...","[0.5866366685377687, 0.3712820091155642, 0.497...","[0.6918483902407707, 0.4856927745072331, 0.143...","[0.0, 0.02936406157968117, 0.14132115411848697...","[0.4385787010938146, 0.1250982116934351, 0.219...","[0.6354445003842665, 0.28561071891161105, 0.62...","[0.8550645060517051, 0.6884280824623885, 0.784...","[1.0, 0.7594701099040868, 0.7428377621977087, ...","[0.43620059742279504, 0.17126361064875048, 0.4...",...,"[0.18263027690560762, 0.0, 0.0, 0.0, 0.1178004...","[0.49198771451120754, 0.3465163390385843, 0.57...","[0.8023176648383595, 0.5875297685548204, 0.454...","[0.4353830465890147, 0.32738553276610755, 0.32...","[0.43761879350299326, 0.17600167677000353, 0.1...","[0.5686543972535026, 0.4575163756832502, 0.606...","[0.749174709350357, 0.5589014746654936, 0.6443...","[0.39508896738104143, 0.22574106630170537, 0.8...","[0.23956509160039643, 0.06386183204540176, 0.3...","[0.16078068136532242, 0.04824031680822762, 0.0..."
2,"[0.43305115456159876, 0.33820919608068983, 0.2...","[0.7272727360986004, 0.4013172120922402, 0.324...","[0.628193005579958, 0.42193623282761633, 0.360...","[0.7348494713934763, 0.45388918362867675, 0.13...","[0.0, 0.0, 0.08854355459381283, 0.242177251600...","[0.4408158590722382, 0.1922719503295613, 0.166...","[0.600681616967235, 0.36074166602101265, 0.504...","[0.8029607741233296, 0.5901930115923396, 0.602...","[1.0, 0.7598807859499533, 0.546100683704346, 0...","[0.42463990910031324, 0.17976566651074732, 0.2...",...,"[0.22602732143513016, 0.0926236454061902, 0.0,...","[0.4947834778732565, 0.3139666541989151, 0.394...","[0.7670635234717356, 0.5989411258878955, 0.319...","[0.5293661385223014, 0.36870989386269004, 0.33...","[0.4536358341616443, 0.3740592212739289, 0.134...","[0.5830647839881333, 0.4106292765966895, 0.430...","[0.7606639062806281, 0.6304157100918507, 0.479...","[0.41593525982356744, 0.22584101950439925, 0.5...","[0.333717696770545, 0.068250233159693, 0.23213...","[0.22648334561385075, 0.054364501723285136, 0...."
3,"[0.3740623804802889, 0.3928083563481559, 0.160...","[0.6973291997292141, 0.5943537959674001, 0.449...","[0.7436668480597128, 0.660162877660154, 0.5590...","[0.6379264861826297, 0.6374250421627957, 0.258...","[0.0, 0.0, 0.031795570390553814, 0.0, 0.001354...","[0.4200321461199312, 0.33768893694259255, 0.17...","[0.6757609382444814, 0.6536756912080274, 0.405...","[0.8007048896273071, 0.8564056310039908, 0.945...","[1.0, 1.0, 0.907485307279899, 0.78238725106086...","[0.4311913649293486, 0.39919134912246734, 0.29...",...,"[0.2232974764757198, 0.21051172045885758, 0.05...","[0.44460813163064694, 0.5084

In [7]:
data = []
for i in range(880):
    list_array = np.array([ast.literal_eval(string) for string in df_norm.iloc[:,i]])
    values = np.concatenate(list_array, axis=0).reshape(32, 8)
    data.append(values)

In [8]:
data[0].shape

(32, 8)

In [9]:
data[0]

array([[0.44854648, 0.2374309 , 0.25671829, 0.13548509, 0.18036478,
        0.41318234, 0.08743186, 0.08291412],
       [0.43767055, 0.25401824, 0.2693243 , 0.32984551, 0.11660465,
        0.29292687, 0.03417741, 0.04475563],
       [0.43305115, 0.3382092 , 0.2658135 , 0.34060688, 0.0820547 ,
        0.22157597, 0.03515119, 0.02825133],
       [0.37406238, 0.39280836, 0.16097329, 0.28599972, 0.10682097,
        0.24319822, 0.03861371, 0.02806835],
       [0.33136929, 0.30007564, 0.17475936, 0.23755972, 0.03357365,
        0.09476266, 0.03789614, 0.0360647 ],
       [0.33137038, 0.18403786, 0.27871445, 0.24270946, 0.0861643 ,
        0.23571907, 0.05857948, 0.0614963 ],
       [0.33796216, 0.41936573, 0.48085898, 0.20918281, 0.09555173,
        0.36320586, 0.0525679 , 0.04218279],
       [0.32646167, 0.23408946, 0.16471522, 0.1750405 , 0.19852092,
        0.0920969 , 0.18221539, 0.16349722],
       [0.27670095, 0.33977189, 0.44620128, 0.25351105, 0.1837161 ,
        0.39400972, 0.117951

## Create train test split

### Data

In [10]:
df_norm.shape[1]

1280

In [11]:
# training with 22 participants each has 40 videos = 880
train = np.zeros((0,32,8))
train[:] = np.nan
#train1 = 
test = np.zeros((0,32,8))
test[:] = np.nan

for i in range(df_norm.shape[1]):
    if i < 880:
        list_array = np.array([ast.literal_eval(string) for string in df_norm.iloc[:,i]])
        values = np.concatenate(list_array, axis=0).reshape(32, 8)
        values = values.reshape(1,32,8)
        train = np.concatenate((train, values), axis=0)
    else:
        list_array = np.array([ast.literal_eval(string) for string in df_norm.iloc[:,i]])
        values = np.concatenate(list_array, axis=0).reshape(32, 8)
        values = values.reshape(1,32,8)
        test = np.concatenate((test, values), axis=0)
    

In [12]:
np.isnan(train).any()

False

In [13]:
np.isnan(test).any()

False

In [14]:
train.shape

(880, 32, 8)

In [15]:
test.shape

(400, 32, 8)

### Get generated Data

### 01 - x2

In [71]:
arrays = []
for i in range(10):
    arrays.append(torch.load("data/sampled/01_fft/a/sampled_fft_12000_128_"+str(i),map_location=torch.device('cpu')))
generated_data = np.concatenate(arrays, axis=0)

In [72]:
generated_data.shape

(1280, 1, 32, 8)

In [73]:
X_argument2 = np.zeros((880,32,8))
X_argument2[:] = np.nan

for i in range(880):
    X_argument2[i] = generated_data[i][0][:,:] 

In [74]:
X_argument2.shape

(880, 32, 8)

In [75]:
# 2x
x_2 = np.concatenate([X_argument2, train], 0) 
print(x_2.shape)

(1760, 32, 8)


### 01 - x3

In [76]:
arrays = []
for i in range(10):
    arrays.append(torch.load("data/sampled/01_fft/b/sampled_fft_12000_128_"+str(i),map_location=torch.device('cpu')))
generated_data = np.concatenate(arrays, axis=0)

In [77]:
generated_data.shape

(1280, 1, 32, 8)

In [78]:
X_argument3 = np.zeros((880,32,8))
X_argument3[:] = np.nan

for i in range(880):
    X_argument3[i] = generated_data[i][0][:,:] 

In [79]:
X_argument3.shape

(880, 32, 8)

In [80]:
# 3x
x_3 = np.concatenate([X_argument2, X_argument3, train], 0) 
print(x_3.shape)

(2640, 32, 8)


### 01 - x4

In [81]:
arrays = []
for i in range(10):
    arrays.append(torch.load("data/sampled/01_fft/c/sampled_fft_12000_128_"+str(i),map_location=torch.device('cpu')))
generated_data = np.concatenate(arrays, axis=0)

In [82]:
generated_data.shape

(1280, 1, 32, 8)

In [83]:
X_argument4 = np.zeros((880,32,8))
X_argument4[:] = np.nan

for i in range(880):
    X_argument4[i] = generated_data[i][0][:,:] 

In [84]:
X_argument2.shape

(880, 32, 8)

In [85]:
# 4x
x_4 = np.concatenate([X_argument2, X_argument3, X_argument4, train], 0) 
print(x_4.shape)

(3520, 32, 8)


### Labels

In [31]:
val22 = valence[: 880]
aro22 = arousal[: 880]
dom22 = dominance[: 880]
lik22 = liking[: 880]

y_test_va = valence[880: ]
y_test_ar = arousal[880: ]
y_test_do = dominance[880: ]
y_test_li = liking[880: ]

In [32]:
# Augmenting the labels 
# 2x
yva2 = np.hstack((val22,val22))
yar2 = np.hstack((aro22,aro22))
ydo2 = np.hstack((dom22,dom22))
yli2 = np.hstack((lik22,lik22))


# 3x
yva3 = np.hstack((val22,val22,val22))
yar3 = np.hstack((aro22,aro22,aro22))
ydo3 = np.hstack((dom22,dom22,dom22))
yli3 = np.hstack((lik22,lik22,lik22))

# 4x
yva4 = np.hstack((val22,val22,val22,val22))
yar4 = np.hstack((aro22,aro22,aro22,aro22))
ydo4 = np.hstack((dom22,dom22,dom22,dom22))
yli4 = np.hstack((lik22,lik22,lik22,lik22))

In [33]:
print(val22.shape)
print(aro22.shape)
print(dom22.shape)
print(lik22.shape)
print(y_test_va.shape)
print(y_test_ar.shape)
print(y_test_do.shape)
print(y_test_li.shape)

(880,)
(880,)
(880,)
(880,)
(400,)
(400,)
(400,)
(400,)


## KNN

In [34]:
train.reshape(-1, 32*8).shape

(880, 256)

In [35]:
# Define KNN based model for actual/original dataset
def knnmodel(xtrain,ytrain,xtest,ytest,kval):
  random_state=100
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtrain.shape,ytrain.shape)
  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval) 
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  #acc = round(acc, 4)
  print("Accuracy",acc)
  # print(classification_report(ytest, y_pred))
  # print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [36]:
# Define KNN based model for augmented data
def knnmodel2(xtrain,ytrain,xtest,ytest,kval):
  
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtest.shape,ytest.shape)

  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval)
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  #acc = round(acc, 4)
  print("Accuracy",acc)
  #print(classification_report(ytest, y_pred))
  #print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [37]:
trainlabel_list = [val22, aro22, dom22, lik22]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
score = []


for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel2(train.reshape(-1, 32*8),traain,test.reshape(-1, 32*8),tesst,kval = k)
    scorelist.append((sc,k))
    score.append(sc)
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4

#for i,name in zip(scorelist,names):
#    print ("Accuracy of ",name ,i)

K value = 1
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 52.5
######################################### Dominance #########################################
Accuracy 61.5
######################################### Liking #########################################
Accuracy 63.0
K value = 2
######################################### Valence #########################################
Accuracy 56.75
######################################### Arousal #########################################
Accuracy 49.0
######################################### Dominance #########################################
Accuracy 53.0
######################################### Liking #########################################
Accuracy 70.5
K value = 3
######################################### Valence #########################################
Accuracy 71.75
#####

Accuracy 64.5
######################################### Liking #########################################
Accuracy 74.0
K value = 20
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.5
######################################### Dominance #########################################
Accuracy 63.0
######################################### Liking #########################################
Accuracy 75.0
K value = 21
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.5
######################################### Dominance #########################################
Accuracy 66.5
######################################### Liking #########################################
Accuracy 74.0
K v

Accuracy 59.0
######################################### Liking #########################################
Accuracy 72.5
K value = 39
######################################### Valence #########################################
Accuracy 74.25
######################################### Arousal #########################################
Accuracy 57.49999999999999
######################################### Dominance #########################################
Accuracy 59.0
######################################### Liking #########################################
Accuracy 70.5
K value = 40
######################################### Valence #########################################
Accuracy 74.25
######################################### Arousal #########################################
Accuracy 53.5
######################################### Dominance #########################################
Accuracy 59.0
######################################### Liking #########################################
Accu

In [ ]:
len(scorelist)

In [38]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(78.25, 27)
(64.5, 20)
(66.5, 21)
(75.0, 16)


### 2X Data

In [39]:
print("Augmented data x2 shape:",x_2.shape)
print(yva2.shape,yar2.shape,ydo2.shape,yli2.shape)

Augmented data x2 shape: (1760, 32, 8)
(1760,) (1760,) (1760,) (1760,)


In [40]:
trainlabel_list = [yva2, yar2, ydo2, yli2]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []

for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    # before x22
    sc = knnmodel2(x_2.reshape(-1, 32*8),traain,test.reshape(-1, 32*8),tesst,kval = k)
    scorelist.append((sc,k))
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4


K value = 1
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 52.5
######################################### Dominance #########################################
Accuracy 61.5
######################################### Liking #########################################
Accuracy 63.0
K value = 2
######################################### Valence #########################################
Accuracy 56.75
######################################### Arousal #########################################
Accuracy 49.0
######################################### Dominance #########################################
Accuracy 53.0
######################################### Liking #########################################
Accuracy 70.5
K value = 3
######################################### Valence #########################################
Accuracy 71.75
#####

Accuracy 75.0
K value = 21
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 63.0
######################################### Dominance #########################################
Accuracy 64.5
######################################### Liking #########################################
Accuracy 75.0
K value = 22
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.5
######################################### Dominance #########################################
Accuracy 62.5
######################################### Liking #########################################
Accuracy 75.0
K value = 23
######################################### Valence #########################################
Acc

Accuracy 74.25
######################################### Arousal #########################################
Accuracy 56.99999999999999
######################################### Dominance #########################################
Accuracy 64.5
######################################### Liking #########################################
Accuracy 70.5
K value = 41
######################################### Valence #########################################
Accuracy 74.25
######################################### Arousal #########################################
Accuracy 57.49999999999999
######################################### Dominance #########################################
Accuracy 64.5
######################################### Liking #########################################
Accuracy 70.5
K value = 42
######################################### Valence #########################################
Accuracy 74.25
######################################### Arousal ###############################

In [41]:
len(scorelist)

196

In [42]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(78.25, 5)
(65.0, 9)
(67.5, 11)
(75.0, 20)


### 3X Data

In [43]:
trainlabel_list = [yva3, yar3, ydo3, yli3]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []

for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel2(x_3.reshape(-1, 32*8),traain,test.reshape(-1, 32*8),tesst,kval = k)
    scorelist.append((sc,k))
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4


K value = 1
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 52.5
######################################### Dominance #########################################
Accuracy 61.5
######################################### Liking #########################################
Accuracy 63.0
K value = 2
######################################### Valence #########################################
Accuracy 56.75
######################################### Arousal #########################################
Accuracy 49.0
######################################### Dominance #########################################
Accuracy 53.0
######################################### Liking #########################################
Accuracy 70.5
K value = 3
######################################### Valence #########################################
Accuracy 71.75
#####

Accuracy 76.75
######################################### Arousal #########################################
Accuracy 63.5
######################################### Dominance #########################################
Accuracy 62.0
######################################### Liking #########################################
Accuracy 75.0
K value = 21
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.0
######################################### Dominance #########################################
Accuracy 62.5
######################################### Liking #########################################
Accuracy 74.0
K value = 22
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.5
#

Accuracy 64.0
######################################### Liking #########################################
Accuracy 68.5
K value = 40
######################################### Valence #########################################
Accuracy 74.75
######################################### Arousal #########################################
Accuracy 55.50000000000001
######################################### Dominance #########################################
Accuracy 64.0
######################################### Liking #########################################
Accuracy 72.5
K value = 41
######################################### Valence #########################################
Accuracy 74.75
######################################### Arousal #########################################
Accuracy 55.50000000000001
######################################### Dominance #########################################
Accuracy 65.0
######################################### Liking #################################

In [44]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(78.25, 29)
(64.5, 17)
(66.0, 44)
(75.0, 20)


### 4X Data

In [55]:
x_4.shape

(3520, 40, 99)

In [45]:
trainlabel_list = [yva4, yar4, ydo4, yli4]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []

for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel2(x_4.reshape(-1, 32*8),traain,test.reshape(-1, 32*8),tesst,kval = k)
    scorelist.append((sc,k))
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4


K value = 1
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 52.5
######################################### Dominance #########################################
Accuracy 61.5
######################################### Liking #########################################
Accuracy 63.0
K value = 2
######################################### Valence #########################################
Accuracy 56.75
######################################### Arousal #########################################
Accuracy 49.0
######################################### Dominance #########################################
Accuracy 53.0
######################################### Liking #########################################
Accuracy 70.5
K value = 3
######################################### Valence #########################################
Accuracy 71.75
#####

Accuracy 63.5
######################################### Liking #########################################
Accuracy 75.0
K value = 20
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.5
######################################### Dominance #########################################
Accuracy 63.0
######################################### Liking #########################################
Accuracy 75.0
K value = 21
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.5
######################################### Dominance #########################################
Accuracy 64.0
######################################### Liking #########################################
Accuracy 75.0
K v

Accuracy 62.5
######################################### Liking #########################################
Accuracy 72.0
K value = 39
######################################### Valence #########################################
Accuracy 75.25
######################################### Arousal #########################################
Accuracy 55.50000000000001
######################################### Dominance #########################################
Accuracy 64.5
######################################### Liking #########################################
Accuracy 68.5
K value = 40
######################################### Valence #########################################
Accuracy 75.25
######################################### Arousal #########################################
Accuracy 54.50000000000001
######################################### Dominance #########################################
Accuracy 62.5
######################################### Liking #################################

In [46]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(78.25, 27)
(64.5, 11)
(67.5, 9)
(75.0, 9)


In [ ]:
np.array_equal(score,scores)

In [ ]:
scorelist = [(15,0),(8,0),(88,0),(40,9),
             (5,6),(67,9),(7,6),(23,0),
             (12,2),(3,1),(5,3),(78,8)]
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4

## SVM

In [47]:
# Define SVM based model for augmented data
def svmmodel(xtrain,ytrain,xtest,ytest,c,kernel = 'linear'):

  # SVM model

  model = SVC(C=c,kernel=kernel,gamma = 'auto')
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  # print(classification_report(ytest, y_pred))
  # print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [48]:
# Using actual data
trainlabel_list = [val22, aro22, dom22, lik22]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(train.reshape(-1, 32*8),traain,test.reshape(-1, 32*8),tesst,c = c)
    scorelist.append(sc)



C value = 0.0001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.01
######################################### Valence #########################################
Ac

Best results <br>
Accuracy of  Valence 59.25 <br>
Accuracy of  Arousal 56.00 <br>
Accuracy of  Dominance 73.0 <br>
Accuracy of  Liking 62.5 <br>

### 2X Data

In [49]:
# Using 2X data
trainlabel_list = [yva2, yar2, ydo2, yli2]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(x_2.reshape(-1, 32*8),traain,test.reshape(-1, 32*8),tesst,c = c)
    scorelist.append(sc)



C value = 0.0001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.01
######################################### Valence #########################################
Ac

Best results <br>
Accuracy of  Valence 59.25 <br>
Accuracy of  Arousal 57.00 <br>
Accuracy of  Dominance 73.00 <br>
Accuracy of  Liking 62.5 <br>

### 3X Data

In [50]:
# Using 2X data
trainlabel_list = [yva3, yar3, ydo3, yli3]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(x_3.reshape(-1, 32*8),traain,test.reshape(-1, 32*8),tesst,c = c)
    scorelist.append(sc)



C value = 0.0001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.01
######################################### Valence #########################################
Ac

Best results <br>
Accuracy of  Valence 59.25 <br>
Accuracy of  Arousal 58.25 <br>
Accuracy of  Dominance 73.25 <br>
Accuracy of  Liking 62.5 <br>

### 4X Data

In [51]:
# Using 2X data
trainlabel_list = [yva4, yar4, ydo4, yli4]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(x_4.reshape(-1, 32*8),traain,test.reshape(-1, 32*8),tesst,c = c)
    scorelist.append(sc)



C value = 0.0001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.01
######################################### Valence #########################################
Ac

Best results <br>
Accuracy of  Valence 59.25 <br>
Accuracy of  Arousal 59.00 <br>
Accuracy of  Dominance 73.5 <br>
Accuracy of  Liking 62.5 <br>

### Neural Network

In [52]:
# Split the data into training/testing sets
def deepmodel(modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size):
  # ()
  
  ytrain = to_categorical(ytrain)
  ytest = to_categorical(ytest)



  input_shape = (32, 8, 1)
  # print(xtrain.shape,xtest.shape)
  xtrain = np.reshape(xtrain,[-1, 32, 8, 1])
  xtest = np.reshape(xtest,[-1, 32, 8, 1])
  print("xtrain.shape,ytrain.shape")
  print(xtrain.shape,ytrain.shape)
  if modelnum == 1:
  # Model 1
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    # model.add(Dense(1, activation='sigmoid'))
  elif modelnum == 2:
    # Resnet kind of model

    def conv_bn_relu(x, filters, kernel_size, strides):
        x = Conv2D(filters=filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        return x


    def identity_block(tensor, filters):
        x = conv_bn_relu(tensor, filters=filters, kernel_size=1, strides=1)
        x = conv_bn_relu(x, filters=filters, kernel_size=3, strides=1)
        x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
        x = BatchNormalization()(x)

        x = Add()([x, tensor])
        x = ReLU()(x)
        return x


    def projection_block(tensor, filters, strides):
        # left branch
        x = conv_bn_relu(tensor, filters=filters, kernel_size=1, strides=1)
        x = conv_bn_relu(x, filters=filters, kernel_size=3, strides=strides)
        x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
        x = BatchNormalization()(x)

        # right branch
        shortcut = Conv2D(filters=4*filters, kernel_size=1, strides=strides)(tensor)  # notice: filters=4*filters
        shortcut = BatchNormalization()(shortcut)

        x = Add()([x, shortcut])
        x = ReLU()(x)
        return x


    def resnet_block(x, filters, reps, strides):
        x = projection_block(x, filters=filters, strides=strides)
        for _ in range(reps-1):  # the -1 is because the first block was a Conv one
            x = identity_block(x, filters=filters)
        return x


    input = Input(shape=(32, 8, 1))

    x = conv_bn_relu(input, filters=64, kernel_size=7, strides=2)  # [3]: 7x7, 64, strides 2
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)  # [3]: 3x3 max mool, strides 2

    x = resnet_block(x, filters=64, reps=3, strides=1)
    x = resnet_block(x, filters=128, reps=4, strides=2)  # s=2 ([2]: conv3_1)
    x = resnet_block(x, filters=256, reps=6, strides=2)  # s=2 ([2]: conv4_1)
    x = resnet_block(x, filters=512, reps=3, strides=2)  # s=2 ([2]: conv5_1)

    x = GlobalAvgPool2D()(x)  # [3]: average pool *it is not written any pool size so we use Global

    # # output = Dense(1, activation='sigmoid')(x)
    output = Dense(2, activation='softmax')(x)
    model = Model(input, output)



  model.compile(loss= 'categorical_crossentropy',#'binary_crossentropy','categorical_crossentropy'
                    optimizer=Adam(),
                    metrics=['accuracy'])
  # Fit data to model
  history = model.fit(xtrain,ytrain ,
            batch_size = batch_size,
            epochs=nepochs,
            verbose=1)

  # Generate generalization metrics
  scores = model.evaluate(xtest, ytest, verbose=0)#verbose =1 
  y_pred = model.predict(xtest)
  #     print("Fold number",fold_no)
  acc = scores[1]*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  print("Loss",scores[0])
  # print(classification_report(ytest.argmax(axis=1), y_pred.argmax(axis=1)))
  # print(confusion_matrix(ytest.argmax(axis=1), y_pred.argmax(axis=1)))
  return acc

## Normal Data

In [86]:
# Using original data
trainlabel_list = [val22, aro22, dom22, lik22]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 200 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,train.reshape(-1, 32,8),traain,test.reshape(-1, 32,8),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)

######################################### Valence #########################################
xtrain.shape,ytrain.shape
(880, 32, 8, 1) (880, 2)
Epoch 1/200
28/28 [==============================] - 1s 7ms/step - loss: 0.6805 - accuracy: 0.5795
Epoch 2/200
28/28 [==============================] - 0s 6ms/step - loss: 0.6255 - accuracy: 0.6375
Epoch 3/200
28/28 [==============================] - 0s 6ms/step - loss: 0.5620 - accuracy: 0.7443
Epoch 4/200
28/28 [==============================] - 0s 6ms/step - loss: 0.5187 - accuracy: 0.7580
Epoch 5/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4915 - accuracy: 0.7818
Epoch 6/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4996 - accuracy: 0.7693
Epoch 7/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4819 - accuracy: 0.7864
Epoch 8/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4864 - accuracy: 0.7795
Epoch 9/200
28/28 [==============================] - 0s 6ms/step 

28/28 [==============================] - 0s 6ms/step - loss: 0.4623 - accuracy: 0.7966
Epoch 82/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4686 - accuracy: 0.7966
Epoch 83/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4634 - accuracy: 0.7977
Epoch 84/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4636 - accuracy: 0.7966
Epoch 85/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4623 - accuracy: 0.7966
Epoch 86/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4661 - accuracy: 0.7920
Epoch 87/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4636 - accuracy: 0.7966
Epoch 88/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4611 - accuracy: 0.7966
Epoch 89/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4656 - accuracy: 0.7966
Epoch 90/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4621 - accuracy: 0.7966
Epoch 91/200


28/28 [==============================] - 0s 7ms/step - loss: 0.4604 - accuracy: 0.7966
Epoch 163/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4595 - accuracy: 0.7966
Epoch 164/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4604 - accuracy: 0.7966
Epoch 165/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4597 - accuracy: 0.7966
Epoch 166/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4624 - accuracy: 0.7966
Epoch 167/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4616 - accuracy: 0.7966
Epoch 168/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4630 - accuracy: 0.7966
Epoch 169/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4615 - accuracy: 0.7966
Epoch 170/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4634 - accuracy: 0.7966
Epoch 171/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4621 - accuracy: 0.7955
Epoc

28/28 [==============================] - 0s 6ms/step - loss: 0.5982 - accuracy: 0.6898
Epoch 42/200
28/28 [==============================] - 0s 6ms/step - loss: 0.5980 - accuracy: 0.6852
Epoch 43/200
28/28 [==============================] - 0s 6ms/step - loss: 0.5981 - accuracy: 0.6943
Epoch 44/200
28/28 [==============================] - 0s 6ms/step - loss: 0.5992 - accuracy: 0.6852
Epoch 45/200
28/28 [==============================] - 0s 7ms/step - loss: 0.5978 - accuracy: 0.6943
Epoch 46/200
28/28 [==============================] - 0s 6ms/step - loss: 0.5969 - accuracy: 0.6864
Epoch 47/200
28/28 [==============================] - 0s 7ms/step - loss: 0.5983 - accuracy: 0.6909
Epoch 48/200
28/28 [==============================] - 0s 6ms/step - loss: 0.5959 - accuracy: 0.6943
Epoch 49/200
28/28 [==============================] - 0s 7ms/step - loss: 0.5986 - accuracy: 0.6864
Epoch 50/200
28/28 [==============================] - 0s 7ms/step - loss: 0.5980 - accuracy: 0.6886
Epoch 51/200


28/28 [==============================] - 0s 7ms/step - loss: 0.5972 - accuracy: 0.6920
Epoch 123/200
28/28 [==============================] - 0s 6ms/step - loss: 0.5942 - accuracy: 0.6932
Epoch 124/200
28/28 [==============================] - 0s 7ms/step - loss: 0.5946 - accuracy: 0.6943
Epoch 125/200
28/28 [==============================] - 0s 6ms/step - loss: 0.5954 - accuracy: 0.6898
Epoch 126/200
28/28 [==============================] - 0s 6ms/step - loss: 0.5953 - accuracy: 0.6909
Epoch 127/200
28/28 [==============================] - 0s 7ms/step - loss: 0.5933 - accuracy: 0.6977
Epoch 128/200
28/28 [==============================] - 0s 7ms/step - loss: 0.5946 - accuracy: 0.6943
Epoch 129/200
28/28 [==============================] - 0s 7ms/step - loss: 0.5954 - accuracy: 0.6909
Epoch 130/200
28/28 [==============================] - 0s 6ms/step - loss: 0.5947 - accuracy: 0.6932
Epoch 131/200
28/28 [==============================] - 0s 6ms/step - loss: 0.5945 - accuracy: 0.6932
Epoc

28/28 [==============================] - 1s 6ms/step - loss: 0.6931 - accuracy: 0.5409
Epoch 2/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6811 - accuracy: 0.5648
Epoch 3/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6759 - accuracy: 0.5852
Epoch 4/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6661 - accuracy: 0.6148
Epoch 5/200
28/28 [==============================] - 0s 8ms/step - loss: 0.6608 - accuracy: 0.5943
Epoch 6/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6398 - accuracy: 0.6500
Epoch 7/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6376 - accuracy: 0.6398
Epoch 8/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6295 - accuracy: 0.6455
Epoch 9/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6227 - accuracy: 0.6614
Epoch 10/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6279 - accuracy: 0.6511
Epoch 11/200
28/28 [=

28/28 [==============================] - 0s 6ms/step - loss: 0.6066 - accuracy: 0.6602
Epoch 84/200
28/28 [==============================] - 0s 6ms/step - loss: 0.6058 - accuracy: 0.6773
Epoch 85/200
28/28 [==============================] - 0s 6ms/step - loss: 0.6045 - accuracy: 0.6716
Epoch 86/200
28/28 [==============================] - 0s 6ms/step - loss: 0.6054 - accuracy: 0.6636
Epoch 87/200
28/28 [==============================] - 0s 6ms/step - loss: 0.6045 - accuracy: 0.6739
Epoch 88/200
28/28 [==============================] - 0s 6ms/step - loss: 0.6049 - accuracy: 0.6727
Epoch 89/200
28/28 [==============================] - 0s 5ms/step - loss: 0.6049 - accuracy: 0.6705
Epoch 90/200
28/28 [==============================] - 0s 6ms/step - loss: 0.6060 - accuracy: 0.6614
Epoch 91/200
28/28 [==============================] - 0s 6ms/step - loss: 0.6043 - accuracy: 0.6750
Epoch 92/200
28/28 [==============================] - 0s 6ms/step - loss: 0.6065 - accuracy: 0.6716
Epoch 93/200


28/28 [==============================] - 0s 7ms/step - loss: 0.6032 - accuracy: 0.6761
Epoch 165/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6039 - accuracy: 0.6795
Epoch 166/200
28/28 [==============================] - 0s 6ms/step - loss: 0.6035 - accuracy: 0.6682
Epoch 167/200
28/28 [==============================] - 0s 6ms/step - loss: 0.6041 - accuracy: 0.6773
Epoch 168/200
28/28 [==============================] - 0s 8ms/step - loss: 0.6032 - accuracy: 0.6750
Epoch 169/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6045 - accuracy: 0.6773
Epoch 170/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6036 - accuracy: 0.6739
Epoch 171/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6031 - accuracy: 0.6739
Epoch 172/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6053 - accuracy: 0.6795
Epoch 173/200
28/28 [==============================] - 0s 7ms/step - loss: 0.6039 - accuracy: 0.6750
Epoc

28/28 [==============================] - 0s 6ms/step - loss: 0.4977 - accuracy: 0.7739
Epoch 44/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4969 - accuracy: 0.7727
Epoch 45/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4954 - accuracy: 0.7716
Epoch 46/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4978 - accuracy: 0.7705
Epoch 47/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4985 - accuracy: 0.7750
Epoch 48/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4971 - accuracy: 0.7727
Epoch 49/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4992 - accuracy: 0.7739
Epoch 50/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4925 - accuracy: 0.7693
Epoch 51/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4954 - accuracy: 0.7705
Epoch 52/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4954 - accuracy: 0.7716
Epoch 53/200


28/28 [==============================] - 0s 7ms/step - loss: 0.4921 - accuracy: 0.7693
Epoch 125/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4951 - accuracy: 0.7727
Epoch 126/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4915 - accuracy: 0.7727
Epoch 127/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4914 - accuracy: 0.7739
Epoch 128/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4935 - accuracy: 0.7727
Epoch 129/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4913 - accuracy: 0.7693
Epoch 130/200
28/28 [==============================] - 0s 7ms/step - loss: 0.4978 - accuracy: 0.7670
Epoch 131/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4928 - accuracy: 0.7693
Epoch 132/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4923 - accuracy: 0.7693
Epoch 133/200
28/28 [==============================] - 0s 6ms/step - loss: 0.4905 - accuracy: 0.7727
Epoc

Best results <br> model 1 with 100 epochs
Accuracy of  Valence 76.75 <br>
Accuracy of  Arousal 64.50 / (63.00) <br>
Accuracy of  Dominance 62.50 / (64.50) / (66.50) / 66.00 <br>
Accuracy of  Liking 74.00 / 75.00) <br>

### 2x Data

In [87]:
# Using 2x data
trainlabel_list = [yva2, yar2, ydo2, yli2]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 200 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,x_2.reshape(-1, 32,8),traain,test.reshape(-1, 32,8),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)

######################################### Valence #########################################
xtrain.shape,ytrain.shape
(1760, 32, 8, 1) (1760, 2)
Epoch 1/200
55/55 [==============================] - 2s 7ms/step - loss: 0.6891 - accuracy: 0.5420
Epoch 2/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6597 - accuracy: 0.5869
Epoch 3/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6240 - accuracy: 0.6267
Epoch 4/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6093 - accuracy: 0.6438
Epoch 5/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5933 - accuracy: 0.6574
Epoch 6/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5883 - accuracy: 0.6625
Epoch 7/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5860 - accuracy: 0.6687
Epoch 8/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5831 - accuracy: 0.6614
Epoch 9/200
55/55 [==============================] - 0s 7ms/ste

55/55 [==============================] - 0s 7ms/step - loss: 0.5629 - accuracy: 0.6909
Epoch 82/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5638 - accuracy: 0.6972
Epoch 83/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5602 - accuracy: 0.6938
Epoch 84/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5631 - accuracy: 0.6903
Epoch 85/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5617 - accuracy: 0.7006
Epoch 86/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5613 - accuracy: 0.6926
Epoch 87/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5641 - accuracy: 0.6903
Epoch 88/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5621 - accuracy: 0.6932
Epoch 89/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5613 - accuracy: 0.7017
Epoch 90/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5628 - accuracy: 0.6983
Epoch 91/200


55/55 [==============================] - 0s 6ms/step - loss: 0.5473 - accuracy: 0.7091
Epoch 163/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5478 - accuracy: 0.7165
Epoch 164/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5476 - accuracy: 0.7182
Epoch 165/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5457 - accuracy: 0.7102
Epoch 166/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5455 - accuracy: 0.7091
Epoch 167/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5451 - accuracy: 0.7108
Epoch 168/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5444 - accuracy: 0.7142
Epoch 169/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5396 - accuracy: 0.7188
Epoch 170/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5412 - accuracy: 0.7170
Epoch 171/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5415 - accuracy: 0.7136
Epoc

55/55 [==============================] - 0s 6ms/step - loss: 0.6252 - accuracy: 0.6540
Epoch 42/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6230 - accuracy: 0.6523
Epoch 43/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6245 - accuracy: 0.6574
Epoch 44/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6266 - accuracy: 0.6557
Epoch 45/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6249 - accuracy: 0.6551
Epoch 46/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6223 - accuracy: 0.6642
Epoch 47/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6248 - accuracy: 0.6602
Epoch 48/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6260 - accuracy: 0.6545
Epoch 49/200
55/55 [==============================] - 0s 8ms/step - loss: 0.6218 - accuracy: 0.6557
Epoch 50/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6223 - accuracy: 0.6676
Epoch 51/200


55/55 [==============================] - 0s 7ms/step - loss: 0.6119 - accuracy: 0.6670
Epoch 123/200
55/55 [==============================] - 0s 8ms/step - loss: 0.6111 - accuracy: 0.6687
Epoch 124/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6132 - accuracy: 0.6659
Epoch 125/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6124 - accuracy: 0.6665
Epoch 126/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6111 - accuracy: 0.6670
Epoch 127/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6115 - accuracy: 0.6682
Epoch 128/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6122 - accuracy: 0.6591
Epoch 129/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6091 - accuracy: 0.6682
Epoch 130/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6117 - accuracy: 0.6676
Epoch 131/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6106 - accuracy: 0.6631
Epoc

55/55 [==============================] - 2s 7ms/step - loss: 0.6915 - accuracy: 0.5341
Epoch 2/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6847 - accuracy: 0.5545
Epoch 3/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6833 - accuracy: 0.5597
Epoch 4/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6821 - accuracy: 0.5688
Epoch 5/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6839 - accuracy: 0.5562
Epoch 6/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6772 - accuracy: 0.5761
Epoch 7/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6726 - accuracy: 0.5864
Epoch 8/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6654 - accuracy: 0.5955
Epoch 9/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6635 - accuracy: 0.5892
Epoch 10/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6577 - accuracy: 0.6028
Epoch 11/200
55/55 [=

55/55 [==============================] - 0s 7ms/step - loss: 0.6334 - accuracy: 0.6398
Epoch 84/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6333 - accuracy: 0.6398
Epoch 85/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6336 - accuracy: 0.6358
Epoch 86/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6318 - accuracy: 0.6494
Epoch 87/200
55/55 [==============================] - 0s 8ms/step - loss: 0.6339 - accuracy: 0.6330
Epoch 88/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6310 - accuracy: 0.6432
Epoch 89/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6317 - accuracy: 0.6483
Epoch 90/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6315 - accuracy: 0.6443
Epoch 91/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6318 - accuracy: 0.6449
Epoch 92/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6336 - accuracy: 0.6375
Epoch 93/200


55/55 [==============================] - 0s 7ms/step - loss: 0.6151 - accuracy: 0.6568
Epoch 165/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6125 - accuracy: 0.6517
Epoch 166/200
55/55 [==============================] - 0s 8ms/step - loss: 0.6226 - accuracy: 0.6420
Epoch 167/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6154 - accuracy: 0.6568
Epoch 168/200
55/55 [==============================] - 0s 8ms/step - loss: 0.6172 - accuracy: 0.6568
Epoch 169/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6138 - accuracy: 0.6545
Epoch 170/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6126 - accuracy: 0.6574
Epoch 171/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6119 - accuracy: 0.6574
Epoch 172/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6177 - accuracy: 0.6545
Epoch 173/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6135 - accuracy: 0.6574
Epoc

55/55 [==============================] - 0s 7ms/step - loss: 0.5539 - accuracy: 0.7312
Epoch 44/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5530 - accuracy: 0.7290
Epoch 45/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5519 - accuracy: 0.7295
Epoch 46/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5527 - accuracy: 0.7290
Epoch 47/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5531 - accuracy: 0.7341
Epoch 48/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5507 - accuracy: 0.7307
Epoch 49/200
55/55 [==============================] - 0s 8ms/step - loss: 0.5490 - accuracy: 0.7358
Epoch 50/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5491 - accuracy: 0.7295
Epoch 51/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5511 - accuracy: 0.7341
Epoch 52/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5517 - accuracy: 0.7295
Epoch 53/200


55/55 [==============================] - 0s 6ms/step - loss: 0.5382 - accuracy: 0.7364
Epoch 125/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5393 - accuracy: 0.7341
Epoch 126/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5380 - accuracy: 0.7347
Epoch 127/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5396 - accuracy: 0.7318
Epoch 128/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5396 - accuracy: 0.7375
Epoch 129/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5381 - accuracy: 0.7307
Epoch 130/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5360 - accuracy: 0.7358
Epoch 131/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5381 - accuracy: 0.7364
Epoch 132/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5363 - accuracy: 0.7341
Epoch 133/200
55/55 [==============================] - 0s 7ms/step - loss: 0.5363 - accuracy: 0.7398
Epoc

Best results <br>
Accuracy of  Valence 57,00 <br>
Accuracy of  Arousal 56.00 <br>
Accuracy of  Dominance 73,00 <br>
Accuracy of  Liking 58,75 <br>

### 3X Data

In [88]:
# Using 2x data
trainlabel_list = [yva3, yar3, ydo3, yli3]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 200 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,x_3.reshape(-1, 32,8),traain,test.reshape(-1, 32,8),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)

######################################### Valence #########################################
xtrain.shape,ytrain.shape
(2640, 32, 8, 1) (2640, 2)
Epoch 1/200
83/83 [==============================] - 2s 7ms/step - loss: 0.6927 - accuracy: 0.5432
Epoch 2/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6680 - accuracy: 0.5803
Epoch 3/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6411 - accuracy: 0.6068
Epoch 4/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6329 - accuracy: 0.6098
Epoch 5/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6266 - accuracy: 0.6227
Epoch 6/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6228 - accuracy: 0.6216
Epoch 7/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6232 - accuracy: 0.6239
Epoch 8/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6230 - accuracy: 0.6189
Epoch 9/200
83/83 [==============================] - 1s 8ms/ste

83/83 [==============================] - 1s 8ms/step - loss: 0.6030 - accuracy: 0.6530
Epoch 82/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6033 - accuracy: 0.6462
Epoch 83/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6040 - accuracy: 0.6470
Epoch 84/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6034 - accuracy: 0.6534
Epoch 85/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6020 - accuracy: 0.6481
Epoch 86/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6032 - accuracy: 0.6511
Epoch 87/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6034 - accuracy: 0.6538
Epoch 88/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6025 - accuracy: 0.6549
Epoch 89/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6022 - accuracy: 0.6534
Epoch 90/200
83/83 [==============================] - 0s 6ms/step - loss: 0.6016 - accuracy: 0.6557
Epoch 91/200


83/83 [==============================] - 1s 6ms/step - loss: 0.5921 - accuracy: 0.6625
Epoch 163/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5937 - accuracy: 0.6602
Epoch 164/200
83/83 [==============================] - 1s 6ms/step - loss: 0.5927 - accuracy: 0.6621
Epoch 165/200
83/83 [==============================] - 1s 6ms/step - loss: 0.5923 - accuracy: 0.6614
Epoch 166/200
83/83 [==============================] - 1s 8ms/step - loss: 0.5915 - accuracy: 0.6633
Epoch 167/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5921 - accuracy: 0.6663
Epoch 168/200
83/83 [==============================] - 1s 6ms/step - loss: 0.5907 - accuracy: 0.6670
Epoch 169/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5903 - accuracy: 0.6629
Epoch 170/200
83/83 [==============================] - 1s 8ms/step - loss: 0.5911 - accuracy: 0.6610
Epoch 171/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5935 - accuracy: 0.6595
Epoc

83/83 [==============================] - 1s 7ms/step - loss: 0.6446 - accuracy: 0.6299
Epoch 42/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6444 - accuracy: 0.6242
Epoch 43/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6452 - accuracy: 0.6284
Epoch 44/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6434 - accuracy: 0.6280
Epoch 45/200
83/83 [==============================] - 1s 8ms/step - loss: 0.6439 - accuracy: 0.6295
Epoch 46/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6436 - accuracy: 0.6288
Epoch 47/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6443 - accuracy: 0.6231
Epoch 48/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6437 - accuracy: 0.6295
Epoch 49/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6437 - accuracy: 0.6242
Epoch 50/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6439 - accuracy: 0.6269
Epoch 51/200


83/83 [==============================] - 1s 7ms/step - loss: 0.6357 - accuracy: 0.6333
Epoch 123/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6359 - accuracy: 0.6299
Epoch 124/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6373 - accuracy: 0.6394
Epoch 125/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6352 - accuracy: 0.6398
Epoch 126/200
83/83 [==============================] - 1s 8ms/step - loss: 0.6349 - accuracy: 0.6341
Epoch 127/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6352 - accuracy: 0.6447
Epoch 128/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6351 - accuracy: 0.6326
Epoch 129/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6363 - accuracy: 0.6379
Epoch 130/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6361 - accuracy: 0.6367
Epoch 131/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6347 - accuracy: 0.6390
Epoc

83/83 [==============================] - 2s 7ms/step - loss: 0.6882 - accuracy: 0.5508
Epoch 2/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6850 - accuracy: 0.5553
Epoch 3/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6835 - accuracy: 0.5545
Epoch 4/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6795 - accuracy: 0.5670
Epoch 5/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6726 - accuracy: 0.5769
Epoch 6/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6700 - accuracy: 0.5803
Epoch 7/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6681 - accuracy: 0.5886
Epoch 8/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6646 - accuracy: 0.5920
Epoch 9/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6671 - accuracy: 0.5958
Epoch 10/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6660 - accuracy: 0.5943
Epoch 11/200
83/83 [=

83/83 [==============================] - 1s 7ms/step - loss: 0.6462 - accuracy: 0.6121
Epoch 84/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6463 - accuracy: 0.6163
Epoch 85/200
83/83 [==============================] - 1s 8ms/step - loss: 0.6463 - accuracy: 0.6117
Epoch 86/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6465 - accuracy: 0.6136
Epoch 87/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6465 - accuracy: 0.6129
Epoch 88/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6464 - accuracy: 0.6182
Epoch 89/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6445 - accuracy: 0.6163
Epoch 90/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6453 - accuracy: 0.6174
Epoch 91/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6443 - accuracy: 0.6182
Epoch 92/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6446 - accuracy: 0.6152
Epoch 93/200


83/83 [==============================] - 1s 7ms/step - loss: 0.6325 - accuracy: 0.6337
Epoch 165/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6318 - accuracy: 0.6356
Epoch 166/200
83/83 [==============================] - 1s 8ms/step - loss: 0.6323 - accuracy: 0.6345
Epoch 167/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6292 - accuracy: 0.6330
Epoch 168/200
83/83 [==============================] - 1s 6ms/step - loss: 0.6332 - accuracy: 0.6311
Epoch 169/200
83/83 [==============================] - 1s 9ms/step - loss: 0.6316 - accuracy: 0.6326
Epoch 170/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6321 - accuracy: 0.6330
Epoch 171/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6323 - accuracy: 0.6273
Epoch 172/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6272 - accuracy: 0.6405
Epoch 173/200
83/83 [==============================] - 1s 7ms/step - loss: 0.6275 - accuracy: 0.6330
Epoc

83/83 [==============================] - 1s 7ms/step - loss: 0.5774 - accuracy: 0.7076
Epoch 44/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5766 - accuracy: 0.7125
Epoch 45/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5776 - accuracy: 0.7095
Epoch 46/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5775 - accuracy: 0.7133
Epoch 47/200
83/83 [==============================] - 1s 8ms/step - loss: 0.5756 - accuracy: 0.7110
Epoch 48/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5759 - accuracy: 0.7144
Epoch 49/200
83/83 [==============================] - 1s 8ms/step - loss: 0.5754 - accuracy: 0.7117
Epoch 50/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5758 - accuracy: 0.7140
Epoch 51/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5742 - accuracy: 0.7144
Epoch 52/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5762 - accuracy: 0.7125
Epoch 53/200


83/83 [==============================] - 1s 9ms/step - loss: 0.5678 - accuracy: 0.7148
Epoch 125/200
83/83 [==============================] - 1s 8ms/step - loss: 0.5666 - accuracy: 0.7155
Epoch 126/200
83/83 [==============================] - 1s 8ms/step - loss: 0.5631 - accuracy: 0.7140
Epoch 127/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5655 - accuracy: 0.7170
Epoch 128/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5639 - accuracy: 0.7163
Epoch 129/200
83/83 [==============================] - 1s 8ms/step - loss: 0.5641 - accuracy: 0.7197
Epoch 130/200
83/83 [==============================] - 1s 8ms/step - loss: 0.5633 - accuracy: 0.7167
Epoch 131/200
83/83 [==============================] - 1s 8ms/step - loss: 0.5630 - accuracy: 0.7208
Epoch 132/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5649 - accuracy: 0.7152
Epoch 133/200
83/83 [==============================] - 1s 7ms/step - loss: 0.5634 - accuracy: 0.7170
Epoc

Best results <br>
Accuracy of  Valence 56.00 <br>
Accuracy of  Arousal 56.50 <br>
Accuracy of  Dominance 67.50 <br>
Accuracy of  Liking 61.0 <br>

### 4X Data

In [89]:
# Using 2x data
trainlabel_list = [yva4, yar4, ydo4, yli4]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 200 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,x_4.reshape(-1, 32,8),traain,test.reshape(-1, 32,8),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)

######################################### Valence #########################################
xtrain.shape,ytrain.shape
(3520, 32, 8, 1) (3520, 2)
Epoch 1/200
110/110 [==============================] - 2s 7ms/step - loss: 0.6884 - accuracy: 0.5352
Epoch 2/200
110/110 [==============================] - 1s 8ms/step - loss: 0.6642 - accuracy: 0.5784
Epoch 3/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6499 - accuracy: 0.5844
Epoch 4/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6481 - accuracy: 0.5963
Epoch 5/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6429 - accuracy: 0.5983
Epoch 6/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6399 - accuracy: 0.5932
Epoch 7/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6388 - accuracy: 0.5977
Epoch 8/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6397 - accuracy: 0.6023
Epoch 9/200
110/110 [==========================

Epoch 80/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6266 - accuracy: 0.6202
Epoch 81/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6262 - accuracy: 0.6273
Epoch 82/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6249 - accuracy: 0.6247
Epoch 83/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6258 - accuracy: 0.6318
Epoch 84/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6250 - accuracy: 0.6324
Epoch 85/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6262 - accuracy: 0.6170
Epoch 86/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6259 - accuracy: 0.6239
Epoch 87/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6249 - accuracy: 0.6321
Epoch 88/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6253 - accuracy: 0.6193
Epoch 89/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6277

110/110 [==============================] - 1s 6ms/step - loss: 0.6045 - accuracy: 0.6528
Epoch 160/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6054 - accuracy: 0.6483
Epoch 161/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6055 - accuracy: 0.6545
Epoch 162/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6014 - accuracy: 0.6551
Epoch 163/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6045 - accuracy: 0.6503
Epoch 164/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6026 - accuracy: 0.6565
Epoch 165/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6020 - accuracy: 0.6560
Epoch 166/200
110/110 [==============================] - 1s 9ms/step - loss: 0.6019 - accuracy: 0.6531
Epoch 167/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6008 - accuracy: 0.6591
Epoch 168/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6016 - a

110/110 [==============================] - 1s 6ms/step - loss: 0.6518 - accuracy: 0.6202
Epoch 37/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6529 - accuracy: 0.6187
Epoch 38/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6544 - accuracy: 0.6168
Epoch 39/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6522 - accuracy: 0.6222
Epoch 40/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6520 - accuracy: 0.6233
Epoch 41/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6522 - accuracy: 0.6213
Epoch 42/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6518 - accuracy: 0.6213
Epoch 43/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6508 - accuracy: 0.6227
Epoch 44/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6515 - accuracy: 0.6216
Epoch 45/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6513 - accuracy: 

110/110 [==============================] - 1s 7ms/step - loss: 0.6405 - accuracy: 0.6293
Epoch 117/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6406 - accuracy: 0.6330
Epoch 118/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6380 - accuracy: 0.6355
Epoch 119/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6393 - accuracy: 0.6301
Epoch 120/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6378 - accuracy: 0.6366
Epoch 121/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6382 - accuracy: 0.6321
Epoch 122/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6374 - accuracy: 0.6378
Epoch 123/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6369 - accuracy: 0.6384
Epoch 124/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6369 - accuracy: 0.6352
Epoch 125/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6380 - a

...

110/110 [==============================] - 1s 7ms/step - loss: 0.6639 - accuracy: 0.5923
Epoch 74/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6631 - accuracy: 0.5912
Epoch 75/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6634 - accuracy: 0.5898
Epoch 76/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6626 - accuracy: 0.5929
Epoch 77/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6644 - accuracy: 0.5849
Epoch 78/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6639 - accuracy: 0.5892
Epoch 79/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6633 - accuracy: 0.5895
Epoch 80/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6629 - accuracy: 0.5923
Epoch 81/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6630 - accuracy: 0.5875
Epoch 82/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6626 - accuracy: 

110/110 [==============================] - 1s 8ms/step - loss: 0.6553 - accuracy: 0.6057
Epoch 153/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6554 - accuracy: 0.6071
Epoch 154/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6545 - accuracy: 0.6017
Epoch 155/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6553 - accuracy: 0.6000
Epoch 156/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6531 - accuracy: 0.6011
Epoch 157/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6559 - accuracy: 0.6014
Epoch 158/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6553 - accuracy: 0.6009
Epoch 159/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6544 - accuracy: 0.6017
Epoch 160/200
110/110 [==============================] - 1s 8ms/step - loss: 0.6545 - accuracy: 0.6068
Epoch 161/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6554 - a

110/110 [==============================] - 1s 6ms/step - loss: 0.5923 - accuracy: 0.7045
Epoch 30/200
110/110 [==============================] - 1s 6ms/step - loss: 0.5913 - accuracy: 0.7060
Epoch 31/200
110/110 [==============================] - 1s 8ms/step - loss: 0.5910 - accuracy: 0.7037
Epoch 32/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5919 - accuracy: 0.7057
Epoch 33/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5917 - accuracy: 0.7037
Epoch 34/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5916 - accuracy: 0.7048
Epoch 35/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5917 - accuracy: 0.7045
Epoch 36/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5906 - accuracy: 0.7063
Epoch 37/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5917 - accuracy: 0.7051
Epoch 38/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5918 - accuracy: 

110/110 [==============================] - 1s 6ms/step - loss: 0.5822 - accuracy: 0.7074
Epoch 110/200
110/110 [==============================] - 1s 6ms/step - loss: 0.5825 - accuracy: 0.7088
Epoch 111/200
110/110 [==============================] - 1s 6ms/step - loss: 0.5832 - accuracy: 0.7057
Epoch 112/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5830 - accuracy: 0.7063
Epoch 113/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5829 - accuracy: 0.7085
Epoch 114/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5833 - accuracy: 0.7077
Epoch 115/200
110/110 [==============================] - 1s 6ms/step - loss: 0.5835 - accuracy: 0.7094
Epoch 116/200
110/110 [==============================] - 1s 6ms/step - loss: 0.5824 - accuracy: 0.7065
Epoch 117/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5830 - accuracy: 0.7074
Epoch 118/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5820 - a

110/110 [==============================] - 1s 6ms/step - loss: 0.5591 - accuracy: 0.7230
Epoch 189/200
110/110 [==============================] - 1s 6ms/step - loss: 0.5574 - accuracy: 0.7244
Epoch 190/200
110/110 [==============================] - 1s 6ms/step - loss: 0.5612 - accuracy: 0.7190
Epoch 191/200
110/110 [==============================] - 1s 8ms/step - loss: 0.5615 - accuracy: 0.7193
Epoch 192/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5598 - accuracy: 0.7247
Epoch 193/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5569 - accuracy: 0.7244
Epoch 194/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5600 - accuracy: 0.7205
Epoch 195/200
110/110 [==============================] - 1s 9ms/step - loss: 0.5582 - accuracy: 0.7247
Epoch 196/200
110/110 [==============================] - 1s 8ms/step - loss: 0.5546 - accuracy: 0.7256
Epoch 197/200
110/110 [==============================] - 1s 7ms/step - loss: 0.5560 - a

Best results <br>
Accuracy of  Valence 52.5 <br>
Accuracy of  Arousal 56.00 <br>
Accuracy of  Dominance 72.25 <br>
Accuracy of  Liking 62.5 <br>